## AND NOW... the Agent Loop!

In [1]:
from agents import Agent, Runner, function_tool
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from rich.console import Console
import docker
import tempfile
import os
import requests
load_dotenv(override=True)

True

In [2]:
todos = []

class ToDoItem(BaseModel):
    description: str = Field(..., description="The text describing the task")
    completed: bool = Field(False, description="Whether the task is complete")

In [4]:
def get_todo_report(print: bool=False) -> str:
    """Get a report of all todos."""
    result = ""
    for index, todo in enumerate(todos):
        completed = "X" if todo.completed else " "
        start = "[strike][green]" if todo.completed else ""
        end = "[/strike][/green]" if todo.completed else ""
        start += "[red]" if "python" in todo.description.lower() else ""
        end += "[/red]" if "python" in todo.description.lower() else ""
        result += f"Todo #{index + 1}: [{completed}] {start}{todo.description}{end}\n"
    if print:
        Console().print(result)
    return result

In [5]:
@function_tool
def create_todos(descriptions: list[str]) -> str:
    """Add new todos from a list of descriptions and return the full list"""
    for desc in descriptions:
        todos.append(ToDoItem(description=desc))
    return get_todo_report(print=True)


@function_tool
def mark_complete(index: int) -> str:
    """Mark complete the todo at the given position (starting from 1) and return the full list"""
    if 1 <= index <= len(todos):
        todos[index - 1].completed = True
    else:
        return "No todo at this index."
    return get_todo_report(print=True)


@function_tool
def list_todos() -> str:
    """Return the full list of todos with completed ones checked off"""
    return get_todo_report()


In [6]:
instructions = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
Provide your solution in Rich console markup (e.g. [bold red]Error[/bold red]) to indicate colors and styles.
"""
tools = [create_todos, mark_complete, list_todos]
agent = Agent("Puzzle Agent", model="gpt-4.1-mini", instructions=instructions, tools=tools)

In [7]:
task = "A train leaves Boston at 2:00 pm traveling 60 mph. Another train leaves New York at 3:00 pm traveling 80 mph toward Boston. When do they meet?"
todos = []
response = await Runner.run(agent, task)
Console().print("\n\n" + response.final_output)

Todo #1: [ ] Determine the distance between Boston and New York
Todo #2: [ ] Calculate the position of the Boston train when the New York train starts at 3:00 pm
Todo #3: [ ] Set up equations for the positions of both trains as functions of time after 3:00 pm
Todo #4: [ ] Solve for the time when the positions of the two trains are equal
Todo #5: [ ] Convert the time back to a clock time for the meeting time

Todo #1: [X] Determine the distance between Boston and New York
Todo #2: [ ] Calculate the position of the Boston train when the New York train starts at 3:00 pm
Todo #3: [ ] Set up equations for the positions of both trains as functions of time after 3:00 pm
Todo #4: [ ] Solve for the time when the positions of the two trains are equal
Todo #5: [ ] Convert the time back to a clock time for the meeting time

Todo #1: [X] Determine the distance between Boston and New York
Todo #2: [X] Calculate the position of the Boston train when the New York train starts at 3:00 pm
Todo #3: [ ] Set up equations for the positions of both trains as functions of time after 3:00 pm
Todo #4: [ ] Solve for the time when the positions of the two trains are equal
Todo #5: [ ] Convert the time back to a clock time for the meeting time

Todo #1: [X] Determine the distance between Boston and New York
Todo #2: [X] Calculate the position of the Boston train when the New York train starts at 3:00 pm
Todo #3: [X] Set up equations for the positions of both trains as functions of time after 3:00 pm
Todo #4: [ ] Solve for the time when the positions of the two trains are equal
Todo #5: [ ] Convert the time back to a clock time for the meeting time

Todo #1: [X] Determine the distance between Boston and New York
Todo #2: [X] Calculate the position of the Boston train when the New York train starts at 3:00 pm
Todo #3: [X] Set up equations for the positions of both trains as functions of time after 3:00 pm
Todo #4: [X] Solve for the time when the positions of the two trains are equal
Todo #5: [ ] Convert the time back to a clock time for the meeting time

Todo #1: [X] Determine the distance between Boston and New York
Todo #2: [X] Calculate the position of the Boston train when the New York train starts at 3:00 pm
Todo #3: [X] Set up equations for the positions of both trains as functions of time after 3:00 pm
Todo #4: [X] Solve for the time when the positions of the two trains are equal
Todo #5: [X] Convert the time back to a clock time for the meeting time

1.107 hours is approximately 1 hour and 6.4 minutes.

Adding this to 3:00 pm:

3:00 pm + 1 hour 6 minutes ≈ 4:06 pm

The two trains will meet at approximately 4:06 pm.

In [8]:
client = docker.from_env()
image = "python:3.12-slim"

DockerException: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [ ]:
@function_tool
def execute_python(code: str) -> str:
    """
    Execute the given Python code inside a Docker container with python:3.12-slim,
    and return whatever is printed to stdout.
    You must print the result of the code to stdout in order to retrieve it.
    This uses the python:3.12-slim image and so it does not have scientific libraries installed;
    write simple python 3.12 code using the standard library only. Do not use numpy or scipy.
    IMPORTANT: You must print the result of the code in order to retrieve it.

    Args:
        code: The Python code to run. Remember to print the result.

    """
    print(f"Executing code: {code}")
    with tempfile.TemporaryDirectory() as tmpdir:
        script_path = os.path.join(tmpdir, "script.py")
        volumes = {tmpdir: {"bind": "/tmp", "mode": "ro"}}
        command = ["python", "/tmp/script.py"]
        with open(script_path, "w") as f:
            f.write(code)
        logs = client.containers.run(image=image, command=command, volumes=volumes, remove=True)
    result = logs.decode("utf-8")
    print(f"Result: {result}")
    return result

In [ ]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

def send_push_notification(message: str):
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

@function_tool
def push(message: str) -> str:
    """Send a text message as a push notification with this brief message

    Args:
        message: The short text message to push
    """

    send_push_notification(message)
    return "Push notification sent"

In [ ]:
instructions = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
You also have access to an execute_python tool to run Python.
To use the execute_python tool, you must have a task on your todo list prefixed with "Write Python code to...".
Write Python code to solve the problem, and then write python to validate your solution to check your work, then use your push tool to send a message with the solution.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution in Rich console markup (e.g. [bold red]Error[/bold red]).
"""
tools = [create_todos, mark_complete, list_todos, execute_python, push]
agent = Agent("Puzzle Agent", model="gpt-4.1-mini", instructions=instructions, tools=tools)

In [ ]:
number = 5 * 11 * 47 * 307
task = f"What are the prime factors of {number}? Reply only with the answer."
todos = []
response = await Runner.run(agent, task)
Console().print("\n\n" + response.final_output)